In [1]:
# Imports

########################################################################
# Python Standard Libraries
import os
import multiprocessing
from timeit import default_timer as timer
import random
import math

########################################################################
# Numpy Library
import numpy as np # linear algebra

########################################################################
# Pandas Library
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

########################################################################
# MATPLOT Library
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.ticker import MaxNLocator
%matplotlib inline

########################################################################
# SKLearn Library
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_recall_curve, classification_report, confusion_matrix, average_precision_score, roc_curve, auc, multilabel_confusion_matrix

########################################################################
# SCIPY Library
from scipy.stats import gaussian_kde
import scipy.stats as st


########################################################################
# Keras Library
from keras.models import Sequential
from keras.layers import Dense

########################################################################
# Init random seed
#seed = 13
#np.random.seed(seed)

In [2]:
# Utility functions
########################################################################
# Print system information
def print_system_info():
    mem_bytes = os.sysconf('SC_PAGE_SIZE') * os.sysconf('SC_PHYS_PAGES')  # e.g. 4015976448
    mem_gib = mem_bytes/(1024.**3)  # e.g. 3.74
    print("{:<23}{:f} GB".format('RAM:', mem_gib))
    print("{:<23}{:d}".format('CORES:', multiprocessing.cpu_count()))
    !lscpu

########################################################################
# Walk through input files
def print_input_files():
    # Input data files are available in the "../input/" directory.
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

########################################################################
# Dump text files
def dump_text_file(fname):
    with open(fname, 'r') as f:
        print(f.read())

########################################################################
# Dump CSV files
def dump_csv_file(fname, count=5):
    # count: 0 - column names only, -1 - all rows, default = 5 rows max
    df = pd.read_csv(fname)
    if count < 0:
        count = df.shape[0]
    return df.head(count)

########################################################################
# Dataset related functions
ds_nbaiot = '/kaggle/input/nbaiot-dataset'
dn_nbaiot = ['Danmini_Doorbell', 'Ecobee_Thermostat', 'Ennio_Doorbell', 'Philips_B120N10_Baby_Monitor', 'Provision_PT_737E_Security_Camera', 'Provision_PT_838_Security_Camera', 'Samsung_SNH_1011_N_Webcam', 'SimpleHome_XCS7_1002_WHT_Security_Camera', 'SimpleHome_XCS7_1003_WHT_Security_Camera']

def fname(ds, f):
    if '.csv' not in f:
        f = f'{f}.csv'
    return os.path.join(ds, f)

def fname_nbaiot(f):
    return fname(ds_nbaiot, f)

def get_nbaiot_device_files():
    nbaiot_all_files = dump_csv_file(fname_nbaiot('data_summary'), -1)
    nbaiot_all_files = nbaiot_all_files.iloc[:,0:1].values
    device_id = 1
    indices = []
    for j in range(len(nbaiot_all_files)):
        if str(device_id) not in str(nbaiot_all_files[j]):
            indices.append(j)
            device_id += 1
    nbaiot_device_files = np.split(nbaiot_all_files, indices)
    return nbaiot_device_files

def get_nbaiot_device_data(device_id, count_norm=-1, count_anom=-1):
    if device_id < 1 or device_id > 9:
        assert False, "Please provide a valid device ID 1-9, both inclusive"
    if count_anom == -1:
        count_anom = count_norm
    device_index = device_id -1
    device_files = get_nbaiot_device_files()
    device_file = device_files[device_index]
    df = pd.DataFrame()
    y = []
    for i in range(len(device_file)):
        fname = str(device_file[i][0])
        df_c = pd.read_csv(fname_nbaiot(fname))
        count = count_anom
        if 'benign' in fname:
            count = count_norm
        rows = count if count >=0 else df_c.shape[0]
        print("processing", fname, "rows =", rows)
        y_np = np.ones(rows) if 'benign' in fname else np.zeros(rows)
        y.extend(y_np.tolist())
        df = pd.concat([df.iloc[:,:].reset_index(drop=True),
                      df_c.iloc[:rows,:].reset_index(drop=True)], axis=0)
    X = df.iloc[:,:].values
    y = np.array(y)
    Xdf = df
    return (X, y, Xdf)

def get_nbaiot_devices_data():
    devices_data = []
    for i in range(9):
        device_id = i + 1
        (X, y) = get_nbaiot_device_data(device_id)
        devices_data.append((X, y))
    return devices_data
#print_input_files()
print_system_info()

RAM:                   18.621841 GB
CORES:                 4
Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  2
Core(s) per socket:  2
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2200.000
BogoMIPS:            4400.00
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0-3
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx

In [3]:
def remove_correlated_features(df, threshold):
    df = df.copy()
    # Create correlation matrix
    corr_matrix = df.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # Find features with correlation greater than a threshold
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

    # Drop features 
    df.drop(to_drop, axis=1, inplace=True)
    return df.iloc[:,:].values

def mark_important_features(vector, pc_keep): # pc_keep is the percentage (0-100) of labels to keep
    th = np.percentile(vector,(100-pc_keep)) # threshold, calculate percentile (100 - percentage) from percentage
    important_bool = (vector >= th)
    important_int = important_bool.astype(int)
    return important_int

def select_features(X, X_norm, X_anom, threshold):
    rows_n = X_norm.shape[0]
    rows_a = X_anom.shape[0]
    if rows_n == 0 or rows_a == 0:
        return X

    y_norm = np.ones(rows_n)
    y_anom = -1 * np.ones(rows_a)

    reg_n = LinearRegression(fit_intercept=False)
    reg_n.fit(X_norm, y_norm)
    coef_n = abs(reg_n.coef_)
    n = mark_important_features(coef_n, threshold)

    reg_a = LinearRegression(fit_intercept=False)
    reg_a.fit(X_anom, y_anom)
    coef_a = abs(reg_a.coef_)
    a = mark_important_features(coef_a, threshold)
   
    mask = np.bitwise_or(n,a)
    mask = mask == 1 # convert to Boolean
    X_sel = X[:, mask]
    return X_sel

In [4]:
def compute_changes(x, y):
    assert x.ndim == 1 and y.ndim == 1, 'Expecting 1 dimension array, received x: {} and y: {}'.format(x.ndim, y.ndim)
    x = x.reshape(-1,1)
    y = y.reshape(-1,1)
    xy = np.column_stack((x,y))
    xy = xy[np.argsort(xy[:, 0])] # sort by x
    changes = 0
    prev_y = None
    for i in range(1, xy.shape[0]):
        y = xy[i][1]
        if y != prev_y:
            prev_y = y
            changes += 1
    return changes

def create_network_structure_dahlia(X, y):
    changes = []
    for i in range(X.shape[1]):
        x = X[:,i]
        change = compute_changes(x,y)
        changes.append(change)
    structure = list(set(changes))
    structure = list(set(np.ceil(np.log(structure))))
    N = X.shape[0]
    structure = [np.floor(math.sqrt(N/2)/s) for s in structure]
    #random.shuffle(structure)
    return structure

def create_network_structure_heuristics(X, y):
    structure = []
    N = X.shape[0]
    m = 1
    node_count_layer_1 = int(math.sqrt((m + 2) * N) + 2 * math.sqrt(N / (m + 2)))
    node_count_layer_2 = int(m * math.sqrt(N / (m + 2)))
    structure.append(node_count_layer_1)
    structure.append(node_count_layer_2)
    return structure

def create_network_structure_genetic(X, y):
    structure = []
    l = 18
    K = 11
    chromosome = ''
    for i in range(l):
        x = random.randint(0, 1)
        chromosome += '{}'.format(x)
    chromosome_left = chromosome[0:K]
    chromosome_right = chromosome[K:]
    #print('chromosome: {}'.format(chromosome))
    #print('split: {} {}'.format(chromosome_left, chromosome_right))
    #print('chromosome_left: {}'.format(chromosome_left))
    #print('chromosome_right: {}'.format(chromosome_right))
    node_count_layer_1 = int(chromosome_left, 2) + random.randint(1, 10)
    node_count_layer_2 = int(chromosome_right, 2) + random.randint(1, 10)
    structure.append(node_count_layer_1)
    structure.append(node_count_layer_2)
    return structure

def create_network_structure_random(X, y):
    layer_count_min = 15
    layer_count_max = 25
    node_count_min = 10
    node_count_max = 97
    
    structure = []
    layer_count = random.randint(layer_count_min, layer_count_max)
    for i in range(layer_count):
        node_count = random.randint(node_count_min, node_count_max)
        structure.append(node_count)
    return structure

In [5]:
def create_binary_classifier(hidden_layers, input_dim):
    layers = []
    for hl in hidden_layers:
        if hl > 0:
            layers.append(hl)

    layer_count = len(layers)
    assert layer_count >= 1, 'at least 1 non-zero hidden layer is needed'
    model = Sequential()
    model.add(Dense(layers[0],input_dim=input_dim,activation='relu'))
    for i in range(1, layer_count):
        model.add(Dense(layers[i],activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    #model.summary()
    model.compile(loss = 'binary_crossentropy',
             optimizer ='adam',metrics=['accuracy'])
    return model

In [6]:
def compute_time_complexity_single_pass(neurons_input, structure, neurons_output):
    count_hidden_layers = len(structure)
    neurons = [neurons_input, *structure, neurons_output]
    complexity = 0
    for i in range(count_hidden_layers + 1):
        complexity += neurons[i] * neurons[i+1]
    return complexity

In [7]:
def compute_report(title, model, X, y):
    y_pred = model.predict(X)
    y_pred[y_pred <= 0] = -1 # convert negative values as 0 for anomaly
    y_pred[y_pred > 0] = 1 # convert positive values as 1 for normal
    acc = accuracy_score(y, y_pred)
    tn, fp, fn, tp = confusion_matrix(y, y_pred, labels=[0,1]).ravel()
    cr = classification_report(y, y_pred)
    print("title,acc,tn,fp,fn,tp")
    print(f'{title}-cm,{acc:.2f},{tn},{fp},{fn},{tp}')
    #print(f'{cr}')
    results = model.evaluate(X, y, verbose=0)
    print(f'{title}-eval,{results}')

def evaluate_different_structures(title, X, y):
    y[y <= 0] = -1 # map negative and 0 as anomaly (-1)
    y[y > 0] = 1 # map positive numbers as normal (1)
    algorithms = [
        {'name': 'Dahlia', 'fx': create_network_structure_dahlia},
        {'name': 'Heuristics', 'fx': create_network_structure_heuristics},
        {'name': 'Genetic', 'fx': create_network_structure_genetic},
        {'name': 'Random', 'fx': create_network_structure_random},
    ]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    print (f"========{title}========")
    feature_count = X.shape[1]
    print(f'Features={feature_count}')
    for algo in algorithms:
        print(f"********{algo['name']}********")
        structure = algo['fx'](X_train,y_train)
        print(f'NN Structure: layers={len(structure)}, neurons: {structure}')
        print('complexity: ', compute_time_complexity_single_pass(feature_count, structure, 1))
        #model = create_binary_classifier(structure, feature_count)
        #model.fit(X_train,y_train,epochs=150,batch_size=10,verbose=0)
        #compute_report('training', model, X_train, y_train)
        #compute_report('validation', model, X_test, y_test)

In [8]:
debug_flag = False

In [9]:
device_to = 9 if not debug_flag else 1
for i in range(device_to):
    device_index = i
    device_id = device_index + 1
    device_name = dn_nbaiot[device_index]
    if not debug_flag:
        (X, y, Xdf) = get_nbaiot_device_data(device_id)
    else:
        (X, y, Xdf) = get_nbaiot_device_data(device_id, 1000, 100)
    X = remove_correlated_features(Xdf, 0.98)
    X_std = StandardScaler().fit_transform(X)
    indices_norm = np.where(y >= 0.5)
    indices_anom = np.where(y <= 0.5)
    X_norm_all = X_std[indices_norm]
    X_anom_all = X_std[indices_anom]
    X_std = select_features(X_std, X_norm_all, X_anom_all,75)
    evaluate_different_structures(device_name, X_std, y)

processing 1.benign.csv rows = 49548
processing 1.gafgyt.combo.csv rows = 59718
processing 1.gafgyt.junk.csv rows = 29068
processing 1.gafgyt.scan.csv rows = 29849
processing 1.gafgyt.tcp.csv rows = 92141
processing 1.gafgyt.udp.csv rows = 105874
processing 1.mirai.ack.csv rows = 102195
processing 1.mirai.scan.csv rows = 107685
processing 1.mirai.syn.csv rows = 122573
processing 1.mirai.udp.csv rows = 237665
processing 1.mirai.udpplain.csv rows = 81982
========Danmini_Doorbell========
Features=47
********Dahlia********
NN Structure: layers=7, neurons: [97.0, 83.0, 73.0, 64.0, 58.0, 53.0, 48.0]
complexity:  32719.0
********Heuristics********
NN Structure: layers=2, neurons: [2384, 476]
complexity:  1247308
********Genetic********
NN Structure: layers=2, neurons: [873, 1]
complexity:  41905
********Random********
NN Structure: layers=21, neurons: [57, 16, 18, 61, 59, 59, 71, 46, 16, 47, 70, 83, 50, 18, 91, 26, 86, 94, 65, 18, 45]
complexity:  57609
processing 2.benign.csv rows = 13113
pr